* Кузовов Сергей Сергеевич группа 54 (дедлайн 13 октября 2021)
* # Заполнены пропуски:
* в колонке "Number of Reviews" значением моды отзывов в городе, где расположен ресторан
* в колонке "Price range" значением моды ('$$ - $$$')
* # Был использован внешний датасет с данными о городах мира "worldcities.csv"
* # Были добавлены признаки:
* Number_of_Reviews_isNAN - наличие пропусков в колонке 'Number of Reviews'
* Cuisine_style_isNAN - наличие пропусков в колонке 'Cuisine style'
* Price_range_isNAN - наличие пропусков в колонке 'Price range'
* Review_isNAN - наличие пропусков в колонке 'Reviews'
* Country - страна
* Population - количество населения в городе
* City_status - статус города как столицы
* cuisine_qnt - количество типов кухни в каждом ресторане
* delta_reviews_date - количество дней прошедших между размещением первого и второго отзыва
* # Было выполнено кодирование категориальных признаков с помощью подхода One-Hot-Encoding:
* Cuisine style 
* Reviews (был использован список наиболее часто встречающихся слов в отзывах о ресторанах)
* City
* Country
* City_status





# import

In [183]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

import re


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [184]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [185]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [186]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [187]:
df_train.info()

In [188]:
df_train.head(5)

In [189]:
df_test.info()

In [190]:
df_test.head(5)

In [191]:
sample_submission.head(5)

In [192]:
sample_submission.info()

In [193]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

****EDA****

In [194]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

К категориальным признак относятся:<br/>
* Price Range<br/>
* City<br/>
* Cuisine Style<br/>
* Reviews<br/>
* Restaurant_id<br/>
* ID_TA<br/>
* URL_TA<br/>

К количественным признакам:
* Ranking
* Rating
* Number of Reviews

In [195]:
data.sample(5)

Признаки, состоящие из нескольких слов

In [196]:
data['Cuisine Style'][1]

In [197]:
data['URL_TA'][1]

In [198]:
data['Reviews'][1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

# Количество и процент пропусков по каждому признаку

In [199]:

nan_df = pd.DataFrame(data.isna().sum(), columns=['Количество'])

nan_df['%'] = nan_df['Количество'].apply(lambda x: round((x/len(data))*100, 0))
print(nan_df)

Откуда следует, что пропуски имеют только 4 признака из 10


# Проверим какое максимальное количество пропусков в строке:

In [200]:

Count_rows = data.apply(lambda x: sum(x.isnull()), axis=1).value_counts()
percent_nans = round(pd.Series(
    Count_rows.index/data.shape[1]*100)).sort_values(ascending=False).astype(str)+' %'
misses = max(data.apply(lambda x: sum(x.isnull()), axis=1))/ data.shape[1]
print('Максимум незаполненных строк в датафрейме:', round(misses*100, 2), "%")
pd.DataFrame({'Количество случаев строке': Count_rows,
              'Количество пропусков в строке': Count_rows.index,
              'Процент незаполненных значений в строке': percent_nans}).sort_values('Количество пропусков в строке', ascending=False).reset_index().drop('index', axis=1)

# Проверим наличие дубликатов по всем строкам

In [201]:
if len(data)>len(data.drop_duplicates()):
    print('Дубликаты есть')
    display(data[data.duplicated()])
else:
    print('Дубликатов нет')



# Обозначим количество пропусков в новых столбцах

In [202]:
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
data['Cuisine_Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')

#  Приведем в порядок колонку Restaurant_id 

In [203]:
data.Restaurant_id = data.Restaurant_id.apply(lambda x: x[3:])
data.Restaurant_id = [int(x) for x in data.Restaurant_id]

# Приведем к числовому формату значение ID в ID_TA

In [204]:
data.ID_TA = data.ID_TA.apply(lambda x: x[1:])
data.ID_TA = [int(x) for x in data.ID_TA]

# Обработаем столбец Cuisine Style

In [205]:
data['Cuisine Style'] = data['Cuisine Style'].astype(str).apply(lambda x: None if x.strip() == '' else x)
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x.replace("['", '').replace("', '", ',').replace("']", ''))

# Обработаем название ресторана из URL

In [206]:
data['URL_TA'] = data['URL_TA'].astype(str).apply(lambda x: None if x.strip() == '' else x)
data['URL_TA'] = data['URL_TA'].apply(lambda x: x.replace("/Restaurant_Review-", '').replace(".html", '').replace('-Reviews-', '|'))
data_m = data['URL_TA'].str.split('|', expand=True)
data['restaurant'] = data_m[1]

#  РАБОТА С ЧИСЛОВЫМИ ПРИЗНАКАМИ #

# 1. Количество отзывов (Number of Reviews)

In [207]:
data['Number of Reviews'].describe()

In [208]:
fig = plt.figure()
axes = fig.add_axes([0, 0, 1, 0.8])
axes.hist(data['Number of Reviews'], bins=100)
axes.set_title('Распределение по количеству отзывов')
axes.set_ylabel('Количество ресторанов')
axes.set_xlabel('Количество отзывов')

# 1. 1 Определяем моду отзывов по городам и заполняем ими пропуски

In [209]:
data['Number of Reviews'] = data.groupby('City')['Number of Reviews'].transform(
    lambda x: x.fillna(round(x.mode(), 0)))

In [210]:
data.info()

2. Место ресторана среди ресторанов своего города (Ranking)

In [211]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

In [212]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

In [213]:
df_train['Ranking'][df_train['City'] == 'London'].hist(bins=100)

Лидеры в топ 10 городов

In [214]:
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Проведем нормализацию признака Ranking в пределах каждого города

In [215]:
means = data.groupby('City')['Ranking'].mean()
std = data.groupby('City')['Ranking'].std()
data['Ranking'] = (data.Ranking - data.City.map(means))/(data.City.map(std))

# 3. Целевая переменная (Rating)

In [216]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

Целевая переменная принимает дискретные значения с шагом 0.5

In [217]:
data['Ranking'][data['Rating'] == 5].hist(bins=100)

In [218]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

Лондон в лидерах)

# КАТЕГОРИАЛЬНЫЕ (НОМИНАТИВНЫЕ) ПРИЗНАКИ

# 1. Диапазон цен в ресторане (Price Range)

In [219]:
data['Price Range'].value_counts()

Кодируем признак с помощью словаря

In [220]:
price_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}
data['Price Range'] = data['Price Range'].replace(to_replace=price_dict)

Заменяем пропуски наиболее часто встречающейся категорией

In [221]:
mode_PR = data['Price Range'].mode()
data['Price Range'] = data['Price Range'].fillna('mode_PR')

# 2. Данные о двух отзывах (Reviews)


In [222]:
data['Reviews'].value_counts()

Обработка колонки Reviews. Замена пустых значений, значением "none".Отсутствие отзывов зафиксируем в отдельную колонку

In [223]:
data['Reviews'] = data.Reviews.apply(lambda x: None if x == '[[], []]' else x)
data['Review_isNAN'] = pd.isna(data['Reviews']).astype('uint8')

Разделим на 2 колонки: содержащий отзывы и содержащий даты

In [224]:
data[['reviews_text', 'reviews_date']
     ] = data['Reviews'].str.split("'],", expand=True)

Выделение даты

In [225]:
data['reviews_date'] = data.reviews_date.dropna().astype(str).apply(
    lambda x: None if pd.isnull(x) else re.compile('\d*/\d*/\d*').findall(x))

Функция для формирования списка даты в определенном формате:

In [226]:
def to_time(line):
    line = [pd.to_datetime(item) for item in line]
    return line
data['reviews_date'] = data.reviews_date.dropna().apply(to_time)

Функция для определения количества дней, которые прошли между последними отзывами:

In [227]:
def find_delta(line):
    return (max(line) - min(line))


data['delta_reviews_date'] = data['reviews_date'].dropna().apply(find_delta).dt.days

Замена пропусков нулевым значением

In [228]:
data['delta_reviews_date'] = data['delta_reviews_date'].fillna(0)

Приведение символов в текстовой колонке (reviews_text) к нижнему регистру

In [229]:
data['reviews_text'] = data.reviews_text.apply(
    lambda x: x if pd.isnull(x) else x.lower())

Выделение из текста слов, которые имеют более двух символов

In [230]:
data['reviews_text_1'] = data.reviews_text.astype(str).apply(
    lambda x: re.compile('[a-z][a-z]\w+').findall(x))

Определение наиболее встречающихся в отзывах выражений

In [231]:
word_list = pd.DataFrame(data.reviews_text_1.dropna(
).tolist()).stack().value_counts().reset_index()
word_list[:40]

Создание списка слов, имеющих эмоциональную окраску для отзыва

In [232]:
words_list = ['not', 'good', 'nice', 'great', 'very', 'best', 'excellent',
              'delicious', 'friendly', 'lovely', 'amazing', 'tasty', 'fantastic', 'average']

Функция, определяющая наиболее часто встречающиеся слова

In [233]:
def check_words(raw):
    line = []
    for item in raw:
        if item in words_list:
            line.append(item)
        else:
            continue
    return line
data['reviews_text_1'] = data['reviews_text_1'].apply(check_words)

# 3. Город расположения ресторана (City)

Использование внешних источников данных для обогащения датасета

In [234]:

df_cities = pd.read_csv('../input/worldcities/worldcities.csv')
df_cities

Исправим назвние 'Porto' к тому, что используется в data - 'Oporto'

In [235]:
df_cities['city_ascii'] = df_cities.city_ascii.apply(
    lambda x: 'Oporto' if x == 'Porto' else x)

Создадим словарь из данных "df_cities", где ключ - город, значение - страна

In [236]:
df_cities_1 = df_cities.drop(
    ['city', 'lat', 'lng', 'iso2', 'iso3', 'admin_name', 'capital', 'population', 'id'], axis=1)
df_countries = df_cities_1[(df_cities_1['country'] != 'United States') & (
    df_cities_1['country'] != 'Canada') & (df_cities_1['country'] != 'Venezuela')]
df_countries.set_index("city_ascii", drop=True, inplace=True)
country_dict = df_countries.to_dict()
country_dict_n = country_dict['country']

Добавление нового признака "Country" (страна)

In [237]:
data['Country'] = data['City'].apply(lambda x: country_dict_n[x])

Создание словаря из данных "df_cities", где ключ - город, значение - размер населения

In [238]:
df_population = df_cities[(df_cities['country'] != 'United States') & (
    df_cities['country'] != 'Canada')]
df_population = df_population.drop(
    ['city', 'lat', 'lng', 'iso2', 'iso3', 'admin_name', 'capital', 'country', 'id'], axis=1)

df_population.set_index("city_ascii", drop=True, inplace=True)
population_dict = df_population.to_dict()
population_dict_n = population_dict['population']

Добавление признака "Population" - население города

In [239]:
data['Population'] = data['City'].apply(lambda x: population_dict_n[x])

Создание сета из названия столиц, используя данные "df_cities"

In [240]:
capitals = set(df_cities[df_cities['capital'] == 'primary']['city_ascii'])

Функция, определяющая является город столицей или нет

In [241]:
def capital_check(city):
    if city in capitals:
        return 'capital' 
    return 'non_capital'

Добавление колонки, с признаком столица

In [242]:
data['Сity_status'] = data['City'].apply(capital_check)

# 4. Блюда кухни (Cuisine style)

Переводим строки к списку

In [243]:
data['Cuisine Style'] = data['Cuisine Style'].astype(str).apply(
    lambda x: str(x).replace('[', '').replace(']', '').replace("'", "").strip())

data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x:  None if x == 'nan' else [
    info.strip() for info in str(x).split(',')])

Список 10 самых популярных кухонь

In [244]:
cuisine_list = pd.DataFrame(data['Cuisine Style'].dropna(
).tolist()).stack().value_counts().reset_index()
top_cuisine = cuisine_list['index'][:10].tolist()

Присваеваем "not_define" отсутствующим значениям

In [245]:
data['Cuisine Style'] = data['Cuisine Style'].apply(
    lambda x: 'not_define' if x == None else x)

Добавляем колонку "Cuisine_qnt", где содержится количество типов кухонь по каждому ресторану

In [246]:
data['cuisine_qnt'] = data['Cuisine Style'].apply(lambda x: len(x))

Функция, оставляющая только популярную кухню

In [247]:
def check_cousine(raw):
    line = []
    top_list = ['Vegetarian Friendly', 'European', 'Mediterranean',
                'Italian', 'Vegan Options', 'Gluten Free Options', 'Bar', 'French', 'Asian']
    for item in raw:
        if item.strip() == 'not_define':
            line.append('not_define')
        elif item.strip() in top_cuisine:
            line.append(item.strip())
        else:
            line.append('other_cuisine')
    return line


data['Cuisine Style'] = data['Cuisine Style'].apply(check_cousine)

Добавляем колонку тип кухни

In [248]:
mlb = MultiLabelBinarizer()
data = data.join(pd.DataFrame(mlb.fit_transform(
    data.pop('Cuisine Style')), index=data.index, columns=mlb.classes_))

# Кодирование категориальных признаков с помощью подхода One-Hot-Encoding

In [249]:
mlb = MultiLabelBinarizer()
data = data.join(pd.DataFrame(mlb.fit_transform(
    data.pop('reviews_text_1')), index=data.index, columns=mlb.classes_))

In [250]:
data = pd.get_dummies(data, columns=['City', 'Country', 'Сity_status'])

# Корреляционный анализ признаков

In [251]:
data_corr = data[data['sample'] == 1]
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data_corr.drop('sample', axis=1).corr())

# Удаление оставшихся категориальных признаков

In [252]:
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis=1, inplace=True)

Функция нормализации данных (не включая : 'Rating','sample')

In [253]:
def StandardScaler_column(d_col):
    scaler = StandardScaler()
    scaler.fit(data[[d_col]])
    return scaler.transform(data[[d_col]])
for i in list(data.columns):
    if i not in ['Rating', 'sample', 'Ranking']:
        data[i] = StandardScaler_column(i)
        if len(data[data[i].isna()]) < len(data):
            data[i] = data[i].fillna(0)

In [254]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [255]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [256]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [257]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [258]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [259]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [260]:
# Как стало известо в результате обработки целевой переменной, реальные рейтинги всегда кратны 0.5
# Напишем функцию соответствующей корректировки предсказанных рейтингов
def fine_rating_pred(rating_pred):
    if rating_pred <= 0.5:
        return 0.0
    if rating_pred <= 1.5:
        return 1.0
    if rating_pred <= 1.75:
        return 1.5
    if rating_pred <= 2.25:
        return 2.0
    if rating_pred <= 2.75:
        return 2.5
    if rating_pred <= 3.25:
        return 3.0
    if rating_pred <= 3.75:
        return 3.5
    if rating_pred <= 4.25:
        return 4.0
    if rating_pred <= 4.75:
        return 4.5
    return 5.0

In [261]:
# Используем округление
for i in range(len(y_pred)):
    y_pred[i] = fine_rating_pred(y_pred[i])

In [262]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [263]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [264]:
test_data.sample(10)

In [265]:
test_data = test_data.drop(['Rating'], axis=1)

In [266]:
sample_submission

In [267]:
predict_submission = model.predict(test_data)

In [268]:
for i in range(predict_submission.size):
        predict_submission[i]=fine_rating_pred(predict_submission[i])

In [269]:
predict_submission

In [270]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
